In [1]:
from google.colab import files
uploaded = files.upload()

Saving amazon2.tsv to amazon2.tsv


In [31]:
!pip install -U textblob
from textblob import TextBlob

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve


In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [3]:
# Load the data
df = pd.read_csv("/content/amazon2.tsv", sep='\t')
df.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [4]:
# Drop rows with missing important values
required_columns = ['rating', 'date', 'verified_reviews', 'variation']
df.dropna(subset=required_columns, inplace=True)

In [5]:
# Fix review text (lowercase handling)
df['verified_reviews'] = df['verified_reviews'].apply(lambda x: ' '.join([word.lower() if word.isupper() else word for word in x.split('\n')]))

In [6]:
# Download nltk resources
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [83]:
# Text cleaning function
def clean_text(text):
    stop_words = set(stopwords.words('english')) - {"not", "no"}
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)


In [90]:
df['verified_reviews'] = df['verified_reviews'].fillna('').astype(str)


In [91]:
# Apply preprocessing
df['cleaned_reviews'] = df['verified_reviews'].apply(preprocess_text)


In [94]:
def extract_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # returns a float between -1 (negative) and 1 (positive)

# Example: Apply to your text data
df['cleaned_reviews'] = df['verified_reviews'].apply(clean_text)


In [97]:
# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=3000, ngram_range=(1, 2))
X = tfidf.fit_transform(df['cleaned_reviews'])
y = df['feedback']

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [99]:
# Neural Network Model (Keras)
nn_model = Sequential()
nn_model.add(Dense(128, activation='relu', input_shape=(X.shape[1],)))
nn_model.add(Dropout(0.3))
nn_model.add(Dense(64, activation='relu'))
nn_model.add(Dropout(0.3))
nn_model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [100]:
# Compile the model
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [101]:
# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=3)


In [103]:
# Train the model
history = nn_model.fit(
    X_train.toarray(), y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test.toarray(), y_test),
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8901 - loss: 0.4969 - val_accuracy: 0.9000 - val_loss: 0.2844
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9295 - loss: 0.1801 - val_accuracy: 0.9000 - val_loss: 0.2307
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9245 - loss: 0.1211 - val_accuracy: 0.9143 - val_loss: 0.2098
Epoch 4/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9805 - loss: 0.0750 - val_accuracy: 0.9222 - val_loss: 0.2261
Epoch 5/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9895 - loss: 0.0401 - val_accuracy: 0.9238 - val_loss: 0.2386
Epoch 6/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9926 - loss: 0.0280 - val_accuracy: 0.9238 - val_loss: 0.2759


In [104]:
# Evaluate the model
loss, test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Final Test Accuracy: {test_accuracy:.4f}")


✅ Final Test Accuracy: 0.9238


In [105]:
# Predict and evaluate
y_probs = nn_model.predict(X_test)
y_pred_nn = (y_probs > 0.5).astype("int32")
print("\nClassification Report (Neural Network):\n", classification_report(y_test, y_pred_nn))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_nn))

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

Classification Report (Neural Network):
               precision    recall  f1-score   support

           0       0.73      0.38      0.50        63
           1       0.93      0.98      0.96       567

    accuracy                           0.92       630
   macro avg       0.83      0.68      0.73       630
weighted avg       0.91      0.92      0.91       630

Confusion Matrix:
 [[ 24  39]
 [  9 558]]


In [106]:
# Print epoch-wise training & validation accuracy
print("\n📈 Epoch-wise Training and Validation Accuracy:")
for i, (train_acc, val_acc) in enumerate(zip(history.history['accuracy'], history.history['val_accuracy']), 1):
    print(f"Epoch {i}: Train Acc = {train_acc:.4f}, Val Acc = {val_acc:.4f}")


📈 Epoch-wise Training and Validation Accuracy:
Epoch 1: Train Acc = 0.9158, Val Acc = 0.9000
Epoch 2: Train Acc = 0.9234, Val Acc = 0.9000
Epoch 3: Train Acc = 0.9313, Val Acc = 0.9143
Epoch 4: Train Acc = 0.9813, Val Acc = 0.9222
Epoch 5: Train Acc = 0.9909, Val Acc = 0.9238
Epoch 6: Train Acc = 0.9933, Val Acc = 0.9238


In [110]:
# Save NN model and vectorizer

nn_model.save("sentiment_nn_model.h5")
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)
print("\n✅ Neural network model and vectorizer saved successfully.")


✅ Neural network model and vectorizer saved successfully.


In [116]:

# Custom review input
user_review = input("Enter a review to analyze: ")

if user_review.strip() != "":
    # Preprocess and compute sentiment
    cleaned_review = preprocess_text(user_review)
    polarity = TextBlob(user_review).sentiment.polarity

    # Vectorize review
    tfidf_vector = vectorizer.transform([cleaned_review]).toarray()

    # Append sentiment score as an additional feature
    input_combined = np.hstack([tfidf_vector, np.array([[polarity]])])

    # Predict
    model_sentiment = model.predict(tfidf_vector)[0][0]  # No sentiment appended
    prediction_label = "Positive 😊" if model_sentiment > 0.5 else "Negative 😞"

    # Output results
    print("\nML Model Prediction:", prediction_label)
    if polarity > 0:
        print(f"TextBlob Polarity: {polarity:.2f} (Positive)")
    elif polarity < 0:
        print(f"TextBlob Polarity: {polarity:.2f} (Negative)")
    else:
        print(f"TextBlob Polarity: {polarity:.2f} (Neutral)")
else:
    print("⚠️ Please enter a valid review.")


Enter a review to analyze: the product is good but not worth it dont buy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

ML Model Prediction: Positive 😊
TextBlob Polarity: 0.27 (Positive)
